In [1]:
# Analyze sentence encodings to make sure nearest neighbors are meaningful

import tensorflow.compat.v1 as tf
import numpy as np
import sys
import os
import glob
import time

sys.path.append("..")
from bert import optimization
from bert import modeling
from bert import tokenization

bert_paths = {
    "rbtl3": "/Users/zifei/Downloads/chinese-bert/chinese_rbtl3_L-3_H-1024_A-16",
    "rbt-ext": "/Users/zifei/Downloads/chinese-bert/chinese_roberta_wwm_ext_L-12_H-768_A-12",
}
bert_dir = bert_paths["rbtl3"]
tokenizer = tokenization.FullTokenizer(bert_dir + "/vocab.txt", do_lower_case=True)

tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [40]:
encodings_file = '/Volumes/WDMPBlue/Files/chinese-bert/poem-data//sent_encodings_all.tsv'

NUM_SENTS = 120000
lineno = 0
sid_to_emb = {}
for line in open(encodings_file):
    lineno += 1
    if lineno > NUM_SENTS:
        break
        
    sid, emb = line.rstrip().split("\t")
    emb = np.array([float(x) for x in emb.split(",")])
    sid_to_emb[int(sid)] = emb

len(sid_to_emb)

120000

In [42]:
from sklearn.neighbors import NearestNeighbors

idxes = []
embeddings = []
for i, e in sid_to_emb.items():
    idxes.append(i)
    embeddings.append(e)
X = np.array(embeddings)
nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(X)

In [43]:
max(idxes), len(idxes)

(120937, 120000)

In [55]:
# some idxes do not have embeddings?
sid_to_sent = {}

max_idxes = max(idxes)
lineno = 0
sents_file = "sents_all.tsv"
for line in open(sents_file):
    lineno += 1
    if lineno > max_idxes:
        break
    sid, pid, sent = line.rstrip().split("\t")
    sid_to_sent[int(sid)] = (pid, sent)

len(sid_to_sent)

120937

In [49]:
len(X)

120000

In [44]:
import random
random.seed(1)

# Similar sents??
sample_size = 30
sample = random.sample(sid_to_emb.keys(), sample_size)

Q = [sid_to_emb[sid] for sid in sample]
all_distances, all_indices = nbrs.kneighbors(Q)

for sid, distances, indices in zip(sample, all_distances, all_indices):
    sent = sid_to_sent[sid]
    print("{}/{}: {}".format(sid, sent[0], sent[1]))
    for d, i in zip(distances, indices):
        idx = idxes[i]
        sent = sid_to_sent[idx]
        print("- {} (idx = {}/{}, dist = {:.3f})".format(sent[1], idx, sent[0], d))
    print("")


17749/1918: 篱前袒腹数飞鸿。
- 篱前袒腹数飞鸿。 (idx = 17749/1918, dist = 0.000)
- 颠连人俟大河清。 (idx = 875/98, dist = 3.260)
- 白蘋洲畔欲移家。 (idx = 98817/11095, dist = 3.413)
- 帆樯两岸阴。 (idx = 79630/8892, dist = 3.663)
- 黄鹄天边并翼难。 (idx = 32909/3626, dist = 3.666)
- 芙蓉蹙沓倚檐多。 (idx = 72727/8105, dist = 3.671)
- 猿狖声多入棹哀。 (idx = 62888/7067, dist = 3.677)
- 画舫移歌入渺瀰。 (idx = 63862/7171, dist = 3.680)
- 长安放衙初岸帻。 (idx = 44824/5018, dist = 3.722)
- 残霭日犹扶。 (idx = 22635/2483, dist = 3.768)

75189/8365: 落花时节每多风。
- 落花时节每多风。 (idx = 75189/8365, dist = 0.000)
- 清醉时时临数行。 (idx = 46847/5247, dist = 4.310)
- 散入重湖半夜风。 (idx = 102508/11502, dist = 4.345)
- 短檠对落花。 (idx = 60988/6848, dist = 4.385)
- 清香每逐好风来。 (idx = 106722/11996, dist = 4.472)
- 春归时听落花风。 (idx = 3276/376, dist = 4.502)
- 几点烟松入幕阴。 (idx = 77341/8621, dist = 4.583)
- 四方风气一腔收。 (idx = 39301/4385, dist = 4.616)
- 秋声不隔树高低。 (idx = 92808/10397, dist = 4.630)
- 四时不断风雷动。 (idx = 77224/8610, dist = 4.648)

111942/12569: 登登山路漫停骖，
- 登登山路漫停骖， (idx = 111942/12569, dist = 0.000)
- 绝壁流云开罨画， (i

In [47]:
# Dist threshold = 4

def print_similar(sample, dist_threshold=4.0):
    Q = [sid_to_emb[sid] for sid in sample]
    all_distances, all_indices = nbrs.kneighbors(Q)

    for sid, distances, indices in zip(sample, all_distances, all_indices):
        if all(d > dist_threshold or d == 0.0 for d in distances):
            continue
        sent = sid_to_sent[sid]
        print("{}/{}: {}".format(sid, sent[0], sent[1]))
        for d, i in zip(distances, indices):
            if d == 0.0:
                continue
            if d > dist_threshold:
                break
            idx = idxes[i]
            sent = sid_to_sent[idx]
            print("- {} (idx = {}/{}, dist = {:.3f})".format(sent[1], idx, sent[0], d))
        print("")


In [52]:
max(sid_to_sent.keys())

120000

In [56]:
random.seed(1)
sample_size = 50
sample = random.sample(sid_to_emb.keys(), sample_size)
print_similar(sample)


17749/1918: 篱前袒腹数飞鸿。
- 颠连人俟大河清。 (idx = 875/98, dist = 3.260)
- 白蘋洲畔欲移家。 (idx = 98817/11095, dist = 3.413)
- 帆樯两岸阴。 (idx = 79630/8892, dist = 3.663)
- 黄鹄天边并翼难。 (idx = 32909/3626, dist = 3.666)
- 芙蓉蹙沓倚檐多。 (idx = 72727/8105, dist = 3.671)
- 猿狖声多入棹哀。 (idx = 62888/7067, dist = 3.677)
- 画舫移歌入渺瀰。 (idx = 63862/7171, dist = 3.680)
- 长安放衙初岸帻。 (idx = 44824/5018, dist = 3.722)
- 残霭日犹扶。 (idx = 22635/2483, dist = 3.768)

105976/11907: 才见黄花放几枝。
- 又见黄花满径开。 (idx = 110710/12452, dist = 3.282)

100892/11336: 秋光祖席送孤蓬。
- 僧舍春寒忆酒旗。 (idx = 31701/3481, dist = 3.947)
- 清燕遥传葛令砂。 (idx = 40987/4576, dist = 3.948)

8336/884: 此道素相期。
- 此日重分茅。 (idx = 31486/3455, dist = 3.912)

15576/1646: 无月无风无雨。
- 无时无处无风雨。 (idx = 12206/1276, dist = 3.920)

100520/11295: 驽骑胜鸣驺。
- 螮蝀欲飞虹。 (idx = 115602/12997, dist = 3.714)
- 俛仰唐虞际。 (idx = 40386/4506, dist = 3.865)

59376/6649: 餐霞几百秋。
- 风送新凉。 (idx = 559/64, dist = 3.826)
- 波晚浴明霞。 (idx = 5507/621, dist = 3.919)
- 天上红云动。 (idx = 96041/10769, dist = 3.967)

62382/7008: 此地几登历。
- 所志在严壑。 (idx =

In [ ]:
# Sample in 40k

In [39]:
import random
random.seed(1)

# Similar sents??
sample_size = 10
sample = random.sample(sid_to_emb.keys(), sample_size)

Q = [sid_to_emb[sid] for sid in sample]
all_distances, all_indices = nbrs.kneighbors(Q)

for sid, distances, indices in zip(sample, all_distances, all_indices):
    sent = sid_to_sent[sid]
    print("{}/{}: {}".format(sid, sent[0], sent[1]))
    for d, i in zip(distances, indices):
        idx = idxes[i]
        sent = sid_to_sent[idx]
        print("- {} (idx = {}/{}, dist = {:.3f})".format(sent[1], idx, sent[0], d))
    print("")


8874/953: 旧盟断比红蕊，
- 旧盟断比红蕊， (idx = 8874/953, dist = 0.000)
- 绕舍乌成阵， (idx = 33313/3682, dist = 4.653)
- 露华向靳金茎下， (idx = 26958/2975, dist = 4.660)
- 两雏双白璧， (idx = 37961/4212, dist = 4.920)
- 履世能销几緉屐， (idx = 13210/1390, dist = 4.994)
- 绿径摇金菊， (idx = 11294/1166, dist = 5.015)
- 金波忽向杯中堕， (idx = 26968/2976, dist = 5.024)
- 两琴颉颃相荡摩， (idx = 16144/1715, dist = 5.066)
- 腐儒此夕南冠坐， (idx = 16038/1696, dist = 5.134)
- 旧札看残烛， (idx = 25829/2839, dist = 5.169)

37595/4172: 谁怜采凤阻千寻。
- 谁怜采凤阻千寻。 (idx = 37595/4172, dist = 0.000)
- 岂让革囊渡。 (idx = 7817/831, dist = 4.408)
- 檐雀啼风未忍闻。 (idx = 1087/132, dist = 4.561)
- 谁似暂幽沉。 (idx = 35185/3896, dist = 4.585)
- 青萍荡碧船自归。 (idx = 8911/954, dist = 4.617)
- 翠袖愁添竹露寒。 (idx = 8424/904, dist = 4.627)
- 暂逢幽赏莫踟蹰。 (idx = 30688/3362, dist = 4.677)
- 谁如朱祐功。 (idx = 21639/2379, dist = 4.680)
- 王孙春草竟如何。 (idx = 31963/3516, dist = 4.683)
- 岂堪攀折赠离人。 (idx = 24435/2671, dist = 4.700)

4168/464: 任重路迢递。
- 任重路迢递。 (idx = 4168/464, dist = 0.000)
- 修褉永和同。 (idx = 18427/2007, dist = 4.068)
- 一剑自

## Sample of similar sentences 

8874/953: 旧盟断比红蕊，
- 旧盟断比红蕊， (idx = 8874/953, dist = 0.000)
- 绕舍乌成阵， (idx = 33313/3682, dist = 4.653)
- 露华向靳金茎下， (idx = 26958/2975, dist = 4.660)
- 两雏双白璧， (idx = 37961/4212, dist = 4.920)
- 履世能销几緉屐， (idx = 13210/1390, dist = 4.994)
- 绿径摇金菊， (idx = 11294/1166, dist = 5.015)
- 金波忽向杯中堕， (idx = 26968/2976, dist = 5.024)
- 两琴颉颃相荡摩， (idx = 16144/1715, dist = 5.066)
- 腐儒此夕南冠坐， (idx = 16038/1696, dist = 5.134)
- 旧札看残烛， (idx = 25829/2839, dist = 5.169)

37595/4172: 谁怜采凤阻千寻。
- 谁怜采凤阻千寻。 (idx = 37595/4172, dist = 0.000)
- 岂让革囊渡。 (idx = 7817/831, dist = 4.408)
- 檐雀啼风未忍闻。 (idx = 1087/132, dist = 4.561)
- 谁似暂幽沉。 (idx = 35185/3896, dist = 4.585)
- 青萍荡碧船自归。 (idx = 8911/954, dist = 4.617)
- 翠袖愁添竹露寒。 (idx = 8424/904, dist = 4.627)
- 暂逢幽赏莫踟蹰。 (idx = 30688/3362, dist = 4.677)
- 谁如朱祐功。 (idx = 21639/2379, dist = 4.680)
- 王孙春草竟如何。 (idx = 31963/3516, dist = 4.683)
- 岂堪攀折赠离人。 (idx = 24435/2671, dist = 4.700)

4168/464: 任重路迢递。
- 任重路迢递。 (idx = 4168/464, dist = 0.000)
- 修褉永和同。 (idx = 18427/2007, dist = 4.068)
- 一剑自能任。 (idx = 39796/4444, dist = 4.112)
- 总负春归。 (idx = 11068/1148, dist = 4.301)
- 抗志罕所同。 (idx = 22445/2470, dist = 4.428)
- 应许仲由仁。 (idx = 16023/1694, dist = 4.437)
- 路途任两歧。 (idx = 8661/924, dist = 4.483)
- 路途任两歧。 (idx = 8334/884, dist = 4.483)
- 总波心易逝。 (idx = 9054/963, dist = 4.565)
- 鲁连安与论。 (idx = 2001/239, dist = 4.570)

16847/1812: 累息有危民。
- 累息有危民。 (idx = 16847/1812, dist = 0.000)
- 星多失地球。 (idx = 16402/1745, dist = 5.335)
- 有所丰收有所贫。 (idx = 37766/4188, dist = 5.413)
- 安危人定。 (idx = 3191/366, dist = 5.479)
- 人事有消亡。 (idx = 29401/3206, dist = 5.511)
- 都缘保障为斯民。 (idx = 32429/3560, dist = 5.533)
- 有约共沧洲。 (idx = 39165/4368, dist = 5.645)
- 消损。 (idx = 10936/1140, dist = 5.665)
- 经年良约负湔裙。 (idx = 7632/821, dist = 5.668)
- 新有脱贫村。 (idx = 12281/1285, dist = 5.689)

7788/830: 不妨唯诺。
- 不妨唯诺。 (idx = 7788/830, dist = 0.000)
- 服药未还童。 (idx = 18156/1980, dist = 5.144)
- 未妨同醉明月。 (idx = 15628/1649, dist = 5.227)
- 于兹莫不同。 (idx = 31809/3494, dist = 5.252)
- 能否换君愉。 (idx = 13897/1472, dist = 5.297)
- 堪问。 (idx = 9210/974, dist = 5.411)
- 旅况复何如。 (idx = 21769/2397, dist = 5.419)
- 不用再相陈。 (idx = 39356/4390, dist = 5.555)
- 轻若未能担。 (idx = 9844/1040, dist = 5.602)
- 经时谢应酬。 (idx = 38767/4312, dist = 5.686)

32722/3600: 冠制喜从周礼后，
- 冠制喜从周礼后， (idx = 32722/3600, dist = 0.000)
- 月下衣冠出， (idx = 26667/2938, dist = 3.876)
- 眼底白云来次第， (idx = 4633/510, dist = 4.142)
- 千里之行始足下， (idx = 22598/2479, dist = 4.195)
- 谈经依壁水， (idx = 24064/2624, dist = 4.257)
- 一片入中心， (idx = 15379/1622, dist = 4.260)
- 夙将三礼魁天下， (idx = 21632/2378, dist = 4.270)
- 明发成何计， (idx = 37323/4142, dist = 4.281)
- 灯负春前约， (idx = 32090/3532, dist = 4.283)
- 共对花前月下时， (idx = 18233/1990, dist = 4.322)

29688/3240: 翠岭连双驿，
- 翠岭连双驿， (idx = 29688/3240, dist = 0.000)
- 淄水穿原绿， (idx = 29420/3208, dist = 4.684)
- 江山晴入画， (idx = 11348/1173, dist = 4.700)
- 岛上田横客， (idx = 31730/3485, dist = 4.715)
- 山川绕红蓼， (idx = 39875/4454, dist = 4.744)
- 九州圆似掌， (idx = 16441/1750, dist = 4.750)
- 寒山到处客双屐， (idx = 30022/3284, dist = 4.756)
- 桃叶渡头三载客， (idx = 24610/2690, dist = 4.768)
- 别馆瀛洲丽， (idx = 33494/3706, dist = 4.794)
- 有鹤南飞新制曲， (idx = 11215/1156, dist = 4.807)

31191/3420: 花边小几听禽坐，
- 花边小几听禽坐， (idx = 31191/3420, dist = 0.000)
- 醉抱白头灯影下， (idx = 29064/3165, dist = 3.962)
- 送目雁边几忘世， (idx = 13202/1389, dist = 4.045)
- 携酒江边吹笛坐， (idx = 32820/3614, dist = 4.088)
- 弯弓月倚阑干上， (idx = 27192/3001, dist = 4.258)
- 手里支藤长过肩， (idx = 36624/4059, dist = 4.287)
- 离堆茗寮外， (idx = 17144/1854, dist = 4.305)
- 偶来空洞坐， (idx = 30867/3386, dist = 4.317)
- 轻寒搁住天边梦， (idx = 13024/1368, dist = 4.346)
- 掩卷山窗下， (idx = 25675/2819, dist = 4.362)

25073/2748: 猿啸惊还乱。
- 猿啸惊还乱。 (idx = 25073/2748, dist = 0.000)
- 薰风萦袖暗相随。 (idx = 1077/130, dist = 4.361)
- 怪蜃嘘云。 (idx = 4319/470, dist = 4.397)
- 竟夕倚秋声。 (idx = 36415/4032, dist = 4.494)
- 黉宇凄迷夜欲阑。 (idx = 9541/1010, dist = 4.560)
- 高春逼暮容。 (idx = 25868/2843, dist = 4.609)
- 两鬓禁愁短又苍。 (idx = 34530/3823, dist = 4.617)
- 心荒劫后沙。 (idx = 10284/1086, dist = 4.653)
- 涛怒太湖狂。 (idx = 12630/1327, dist = 4.729)
- 涛怒乱帆收。 (idx = 36667/4066, dist = 4.739)

13867/1469: 北照紫荆关。
- 北照紫荆关。 (idx = 13867/1469, dist = 0.000)
- 南客话寒天。 (idx = 26551/2922, dist = 3.385)
- 分水向彭城。 (idx = 38424/4269, dist = 3.634)
- 春星照剑明。 (idx = 39676/4428, dist = 3.756)
- 林下清溪冷。 (idx = 8467/909, dist = 3.758)
- 汤汤下海东。 (idx = 12339/1293, dist = 3.764)
- 五老集西郊。 (idx = 14390/1516, dist = 3.771)
- 黄叶客途赊。 (idx = 31142/3414, dist = 3.790)
- 同归竹林里。 (idx = 33060/3644, dist = 3.819)
- 霜月倚栏干。 (idx = 8358/888, dist = 3.828)
